In [9]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

In [10]:
from typing import List

class MSExperimentDF(MSExperiment):
    def __init__(self):
        super().__init__()

    ## TODO add chromatogram version
    ## TODO metadata df?

    ## TODO add spectrum_reference, add ion_mobility, add MS level
    def to_df(self, melt : bool = False):
        if melt:
            spectraarr = np.fromiter(((spec.getRT(), point[0], point[1]) for spec in self for point in zip(*spec.get_peaks())), dtype=[('RT', 'f'), ('mz', 'f'), ('inty', 'f')])#, count=sum(s.size() for s in exp))
            # Initial tests showed that loading into numpy array first is faster than direct construction from iter.
            return pd.DataFrame(data=spectraarr)
        else:
            cols = ["RT", "mzarray", "intarray"]
            return pd.DataFrame(data=((spec.getRT(), *spec.get_peaks()) for spec in self), columns=cols)

In [11]:
exp = MSExperimentDF()
MzMLFile().load("BSA1_F1.mzML", exp)


In [12]:
display(exp.to_df(True))

,RT,mz,inty
0,1501.413940,300.089752,3431.026123
1,1501.413940,300.181335,1181.808960
2,1501.413940,300.202667,1516.174561
3,1501.413940,300.298279,1719.854736
4,1501.413940,300.334595,1115.259644
...,...,...,...
193055,1999.839111,600.408447,164.921005
193056,1999.839111,601.383606,45.095955
193057,1999.839111,607.242554,1.255427
193058,1999.839111,610.389587,13.149967


In [13]:
display(exp.to_df(False))

,RT,mzarray,intarray
0,1501.413940,"[300.0897645621494, 300.18132740129533, 300.20...","[3431.0261, 1181.809, 1516.1746, 1719.8547, 11..."
1,1503.031250,"[300.06577092599525, 300.08932376441896, 300.2...","[914.79034, 1842.2311, 2395.1025, 851.4738, 16..."
2,1504.315186,"[300.0655376904414, 300.08949705724626, 300.20...","[789.3734, 2810.2341, 1751.1084, 1107.8386, 14..."
3,1506.103271,"[300.08969729528235, 300.14442951270433, 300.1...","[3358.1047, 1391.3488, 2010.7505, 3580.989, 16..."
4,1507.691284,"[300.02961474114085, 300.0896262060012, 300.14...","[1056.8984, 2595.467, 1318.1053, 3188.7034, 13..."
...,...,...,...
762,1996.676147,"[206.9207000732422, 239.19674682617188, 253.32...","[6.038003, 1.8395934, 4.4162316, 2.2767653, 9...."
763,1997.912720,"[89.19850158691406, 96.14889526367188, 97.1662...","[2.2128685, 1.5487204, 12.895538, 8.870416, 18..."
764,1998.262329,"[136.0441131591797, 140.91648864746094, 142.10...","[8.844822, 10.610469, 48.748848, 2.2722573, 2...."
765,1998.608398,"[152.00709533691406, 184.16302490234375, 198.2...","[4.3010125, 4.11401, 5.2848134, 1.5501136, 16...."
